<a href="https://colab.research.google.com/github/janwaelty/Master-Thesis/blob/main/visual_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Networks and the Art Market: A Deep Learning Approach to Valuation**


In [22]:

import os
import shutil
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import re
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from scipy.stats import linregress
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torchvision import datasets, transforms
import shutil
import os
from google.colab import drive
import zipfile

# Define source and destination paths
source_path = "/content/drive/My Drive/Colab Notebooks/visual_model.ipynb"
destination_path = "/content/drive/My Drive/visual_model.ipynb"

  # Check if the source file exists before moving
if os.path.exists(source_path):
  # Move the file
  shutil.move(source_path, destination_path)
  print(f"Notebook moved to: {destination_path}")

!cp /content/drive/My\ Drive/data_preprocess.py /content/

from data_preprocess import (
    data_split as data_split,
    fit_visualization as fit_visualization,
    repeated_to_single_transactions as repeated_to_single_transactions,
    one_hot as one_hot,
    standardization as standardization,
    data_filter as data_filter,
    add_prev_avg_price as add_prev_avg_price,
    standardization as standardization
)


!cp /content/drive/My\ Drive/image_preprocess.py /content/

from image_preprocess import (
    load_image_data as load_image_data,
    check_image_existence as check_image_existence,
    filter_data_for_missing_images as filter_data_for_missing_images,
    resnet_transform as resnet_transform
)







Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from PIL import Image
import torch
from torchvision import transforms
import os



class MixedTransactionDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, base_dir,, transform=None):
        # Ensure numerical data is in float32 format
        self.X = X.reset_index(drop=True).values.astype('float32')  # Convert X to numpy array
        self.y = y.reset_index(drop=True).values.astype('float32')  # Convert y to numpy array
        self.image_paths = [os.path.join(base_dir, path.lstrip('/')) for path in X['image_url'].dropna()]
        self.X = X.drop(columns=['image_url']).reset_index(drop=True).values.astype('float32')
        self.transform = transform

    def __getitem__(self, index):
        # Get numerical features (X) and target values (y)
        x = torch.tensor(self.X[index], dtype=torch.float32)
        y = torch.tensor(self.y[index], dtype=torch.float32)

        # Get the image path (image_url) and open the image
        image_path = self.image_paths[index]
        image = Image.open(image_path)

        # Apply any transformations if provided (resize, normalization, etc.)
        if self.transform:
            image = self.transform(image)

        return x, y, image

    def __len__(self):
        return len(self.X)



def create_dataloaders(X_train, X_test, y_train, y_test, base_dir,  batch_size=32):
    # Create the training dataset and dataloader
    train_dataset = MixedTransactionDataset(X_train, y_train, transform=resnet_transform())
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    # Create the testing dataset and dataloader
    test_dataset = MixedTransactionDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader


**Load zip file with images**

In [ ]:
if __name__ == 'main':
  # Mount Google Drive
  drive.mount('/content/drive')
  # Define source and destination paths
  source_path = "/content/drive/My Drive/Colab Notebooks/visual_model.ipynb"
  destination_path = "/content/drive/My Drive/visual_model.ipynb"

  # Check if the source file exists before moving
  if os.path.exists(source_path):
    # Move the file
    shutil.move(source_path, destination_path)
    print(f"Notebook moved to: {destination_path}")

  artist_data_path= "/content/drive/My Drive/artist_graph_data.xlsx"
  artwork_numeric_data = pd.read_excel(artist_data_path)


  selected_vars =  ['category', 'artist', 'transaction_price', 'height',
                  'width', 'medium', 'transaction_house',
                    'transaction_year_semi', 'image_url']
  one_hot_vars = ['category', 'artist', 'medium', 'transaction_house']



  X_train, X_test, y_train, y_test = data_split(artwork_numeric_data,
                                                   selected_vars,
                                                   one_hot_vars, 2020)
